In [ ]:
import { load } from "@std/dotenv";

import { z } from '@zod/zod';


import { generateBrandInfo, generateCompetitorsInfo, concatBrands } from "../src/brands.ts";
import { generatePersonas } from "../src/personas.ts";
import { generateFunnel, iterateFunnelCategories } from "../src/funnel.ts";
import { ModelId } from "../src/models.ts";
import { searchBatch, searchWithFormat } from "../src/search.ts";
import { extractABSForBrandBatch } from "../src/sentiment.ts";
import { Brand } from "../src/schemas/brand.schema.ts";
import { translateBatch } from "../src/translate.ts";
import { audit, enrichAudit } from "../src/audit.ts";

In [2]:
void await load({
  envPath: "../../.env",
  export: true,
});


# Setup

In [3]:
const config = {
  brand: "Peugeot.es",
  sector: "coches eléctricos",
  country: "es",
  language: "es",
  models: ["openai/gpt-4.1-mini"],  //, "google/ai-overview"],
  numPersonas: 5,
  personaModel: "gpt-4.1",
  funnelModel: "gpt-4.1",
};

let start: number;
let duration: string;

# Brands

Market & sector from brand domain

In [4]:
start = Date.now();

const brand = await generateBrandInfo({
  brand: config.brand,
  language: config.language,
  model: 'gpt-4.1',
  useSearch: true,
  contextSize: 'high',
});

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`Generated competitors in ${duration}s`);
console.log(JSON.stringify(brand, null, 2));

Generated competitors in 12.0s
{
  "name": "Peugeot España S.A.",
  "shortName": "Peugeot",
  "description": "Fabricante y comercializador de automóviles, incluyendo turismos, vehículos comerciales y deportivos.",
  "domain": "https://www.peugeot.es",
  "sectors": [
    "Automoción",
    "Vehículos comerciales",
    "Vehículos deportivos"
  ],
  "markets": [
    "España",
    "Europa",
    "América Latina",
    "Asia"
  ],
  "portfolio": [
    {
      "name": "Peugeot 208",
      "category": "Turismo compacto"
    },
    {
      "name": "Peugeot 3008",
      "category": "SUV"
    },
    {
      "name": "Peugeot Partner",
      "category": "Vehículo comercial"
    },
    {
      "name": "Peugeot e-208",
      "category": "Vehículo eléctrico"
    }
  ],
  "marketPosition": "leader",
  "favicon": "https://www.peugeot.es/favicon.ico"
}


In [5]:
start = Date.now();

const competitors = await generateCompetitorsInfo({
  brand: config.brand,
  sector: config.sector,
  market: config.country,
  strict: true,
  language: config.language,
  model: 'gpt-4.1',
  useSearch: true,
  contextSize: 'high',
});

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`Generated competitors in ${duration}s`);
console.log(competitors[0]);

Generated competitors in 50.9s
{
  name: "Renault",
  shortName: "Renault",
  description: "Fabricante francés de automóviles con una amplia gama de vehículos eléctricos.",
  domain: "https://www.renault.es",
  sectors: [ "Automoción" ],
  markets: [ "Europa", "España" ],
  portfolio: [
    { name: "Renault ZOE", category: "Coche eléctrico" },
    { name: "Renault Twingo E-Tech", category: "Coche eléctrico" },
    { name: "Renault Mégane E-Tech", category: "Coche eléctrico" }
  ],
  marketPosition: "leader",
  favicon: "https://www.renault.es/favicon.ico"
}


In [6]:
const all_brands : Brand[] = concatBrands([brand], competitors);
all_brands

[
  {
    name: "Peugeot España S.A.",
    shortName: "Peugeot",
    description: "Fabricante y comercializador de automóviles, incluyendo turismos, vehículos comerciales y deportivos.",
    domain: "https://www.peugeot.es",
    sectors: [ "Automoción", "Vehículos comerciales", "Vehículos deportivos" ],
    markets: [ "España", "Europa", "América Latina", "Asia" ],
    portfolio: [
      { name: "Peugeot 208", category: "Turismo compacto" },
      { name: "Peugeot 3008", category: "SUV" },
      { name: "Peugeot Partner", category: "Vehículo comercial" },
      { name: "Peugeot e-208", category: "Vehículo eléctrico" }
    ],
    marketPosition: "leader",
    favicon: "https://www.peugeot.es/favicon.ico",
    isCompetitor: false
  },
  {
    name: "Renault",
    shortName: "Renault",
    description: "Fabricante francés de automóviles con una amplia gama de vehículos eléctricos.",
    domain: "https://www.renault.es",
    sectors: [ "Automoción" ],
    markets: [ "Europa", "España" ],
 

 # Personas


In [7]:
start = Date.now();
const personas = await generatePersonas({
  sector: config.sector,
  market: config.country,
  brand: config.brand,
  language: "spanish",
  count: config.numPersonas,
}
);
duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`Generated personas in ${duration}s`);
console.log(personas);

Generated personas in 17.2s
[
  {
    name: "Padre Ahorrador Urbano",
    description: "Padre de familia de entre 35 y 50 años que reside en grandes ciudades y busca eficiencia, seguridad y ahorro a la hora de cambiar de coche familiar. Le interesa la movilidad sostenible y desea reducir los costes asociados al uso del coche tradicional, especialmente en desplazamientos diarios. Evalúa mucho la relación calidad-precio y valora la facilidad de carga, autonomía y facilidades para familias.",
    keywordSeeds: [
      "mejor coche eléctrico familiar",
      "coches eléctricos para familias",
      "autonomía coche eléctrico",
      "ahorro coche eléctrico",
      "coche eléctrico carga rápida",
      "comparativa coches eléctricos",
      "seguridad coche eléctrico",
      "maletero grande coche eléctrico"
    ]
  },
  {
    name: "Joven Profesional Eco-Consciente",
    description: "Profesional joven, entre 25 y 35 años, con alto nivel educativo y conciencia medioambiental. Vive en ciuda

# Funnel

In [8]:
start = Date.now();
const funnel = await generateFunnel({
  sector: config.sector,
  language: config.language,
  country: config.country,
});
duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`✅ Funnel customized and seeded in ${duration}s!\n`);
for (const [stage, goal, category] of iterateFunnelCategories(funnel)) {
    console.log(`Stage: ${stage}, Goal: ${goal}, Category: ${category.name}, numSeeds: ${category.examples.length}`);
}

✅ Funnel customized and seeded in 39.0s!

Stage: Conciencia / Descubrimiento, Goal: Reconocimiento del problema, educación., Category: Identificación del Problema, numSeeds: 10
Stage: Conciencia / Descubrimiento, Goal: Reconocimiento del problema, educación., Category: Educación sobre la Categoría, numSeeds: 10
Stage: Conciencia / Descubrimiento, Goal: Reconocimiento del problema, educación., Category: Tendencias & Inspiración, numSeeds: 10
Stage: Consideración / Investigación, Goal: Comparar opciones, evaluar soluciones., Category: Características & Especificaciones, numSeeds: 10
Stage: Consideración / Investigación, Goal: Comparar opciones, evaluar soluciones., Category: Comparativas, numSeeds: 10
Stage: Consideración / Investigación, Goal: Comparar opciones, evaluar soluciones., Category: Idoneidad y Casos de Uso, numSeeds: 10
Stage: Consideración / Investigación, Goal: Comparar opciones, evaluar soluciones., Category: Opiniones & Reseñas, numSeeds: 10
Stage: Decisión / Evaluación, 

# Keywords

In [ ]:
import { KeywordRecord } from "../lib/GoogleAds/keywordPlanner.ts";
import { expandKeywords } from "../src/keywords.ts";

start = Date.now();

const sectorSeeds = [config.sector];
const personaSeeds = personas.map(p => p.keywordSeeds);
const funnelSeeds = iterateFunnelCategories(funnel)
    .map(([_stage, _goal, category]) => category.examples)
    .toArray();

const allSeeds = [
	...personaSeeds,
	...funnelSeeds,
	sectorSeeds
];

const { filePath, description } = await expandKeywords({
	seedKeywords: allSeeds,
	url: config.brand,
	language: config.language,
	countryISOCode: config.country
});

In [ ]:
import { readParquetRows } from "./readParquetRows.ts";

const keywords = await readParquetRows<KeywordRecord>(filePath);

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`\n✅ Total: Expanded to ${keywords.length} unique keywords in ${duration}s\n`);


✅ Total: Expanded to 3753 unique keywords in 189.2s



In [12]:
keywords[0]

{
  ROWID: 1,
  keyword: "mejor coche eléctrico pequeño",
  avgMonthlySearches: 170,
  competition: "HIGH",
  competitionIndex: 95,
  averageCpc: 0.6685150265693665,
  lowTopOfPageBid: 0.2526960074901581,
  highTopOfPageBid: 0.9091740250587463,
  searchVolume: [
    110, 140, 140, 170,
    140, 170, 110, 110,
    170, 170, 210, 260
  ],
  searchVolumeDate: [
    2024-10-01T00:00:00.000Z,
    2024-11-01T00:00:00.000Z,
    2024-12-01T00:00:00.000Z,
    2025-01-01T00:00:00.000Z,
    2025-02-01T00:00:00.000Z,
    2025-03-01T00:00:00.000Z,
    2025-04-01T00:00:00.000Z,
    2025-05-01T00:00:00.000Z,
    2025-06-01T00:00:00.000Z,
    2025-07-01T00:00:00.000Z,
    2025-08-01T00:00:00.000Z,
    2025-09-01T00:00:00.000Z
  ],
  searchVolumeGrowthYoy: 136.36363220214844,
  searchVolumeGrowth3m: 52.94117736816406,
  searchVolumeGrowth1m: 23.809524536132812,
  searchVolumeTrend: 0.05167464166879654,
  concepts: null,
  conceptGroups: null
}

# Translate to Prompts 

In [13]:
let kwds = keywords.map(k => k.keyword);
let prompts = await translateBatch(kwds, config.language, 'gpt-4.1-mini');

In [14]:
prompts[0]

"¿Cuál es el mejor coche eléctrico pequeño?"

# Cache generated audit inputs

In [15]:
const cache = {
    brand,
    competitors,
    funnel,
    personas,
    keywords,
    prompts
};

await Deno.writeTextFile("./cache.json", JSON.stringify(cache, null, 2));
console.log("✅ Cache saved to ./cache.json");

✅ Cache saved to ./cache.json


In [16]:
const cached = JSON.parse(await Deno.readTextFile("./cache.json"));
console.log("✅ Cache loaded from ./cache.json");

✅ Cache loaded from ./cache.json


# Prepare sample for audit

In [17]:
// Enrich keywords with translated prompts
const kwdPrompts = cached.keywords.map((keyword, index) => ({
	...keyword,
	prompt: cached.prompts[index]
}));

console.log(`✅ Enriched ${kwdPrompts.length} keywords with prompts`);
console.log("Example:", kwdPrompts[0]);

✅ Enriched 3753 keywords with prompts
Example: {
  ROWID: 1,
  keyword: "mejor coche eléctrico pequeño",
  avgMonthlySearches: 170,
  competition: "HIGH",
  competitionIndex: 95,
  averageCpc: 0.6685150265693665,
  lowTopOfPageBid: 0.2526960074901581,
  highTopOfPageBid: 0.9091740250587463,
  searchVolume: [
    110, 140, 140, 170,
    140, 170, 110, 110,
    170, 170, 210, 260
  ],
  searchVolumeDate: [
    "2024-10-01T00:00:00.000Z",
    "2024-11-01T00:00:00.000Z",
    "2024-12-01T00:00:00.000Z",
    "2025-01-01T00:00:00.000Z",
    "2025-02-01T00:00:00.000Z",
    "2025-03-01T00:00:00.000Z",
    "2025-04-01T00:00:00.000Z",
    "2025-05-01T00:00:00.000Z",
    "2025-06-01T00:00:00.000Z",
    "2025-07-01T00:00:00.000Z",
    "2025-08-01T00:00:00.000Z",
    "2025-09-01T00:00:00.000Z"
  ],
  searchVolumeGrowthYoy: 136.36363220214844,
  searchVolumeGrowth3m: 52.94117736816406,
  searchVolumeGrowth1m: 23.809524536132812,
  searchVolumeTrend: 0.05167464166879654,
  concepts: null,
  conceptGro

In [18]:
function randomSample<T>(array: Array<T>, n: number): Array<T> {
	if (n >= array.length) {
		return [...array];
	}

	const shuffled = [...array];
	for (let i = shuffled.length - 1; i > 0; i--) {
		const j = Math.floor(Math.random() * (i + 1));
		[shuffled[i], shuffled[j]] = [shuffled[j], shuffled[i]];
	}

	return shuffled.slice(0, n);
}

// Example: sample 10 random keyword-prompt pairs
const sample = randomSample(kwdPrompts, 400);

# Test audit

In [19]:
let samplePrompts = sample.map(s => s.prompt);

start = Date.now();

const auditResults = await audit({
	prompts: samplePrompts,
	models: config.models,
	brand: cached.brand,
	competitors: cached.competitors,
	useSearch: true,
});

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`Audit completed in ${duration}s\n`)
console.log(`Results: ${auditResults.length} search results with flagged sources\n`);

Audit completed in 32.5s

Results: 400 search results with flagged sources



In [20]:
console.log(JSON.stringify(auditResults[0], null, 2));

{
  "answer": "Los coches de hidrógeno, aunque ofrecen ventajas como cero emisiones y tiempos de repostaje rápidos, suelen tener precios más elevados que los vehículos convencionales. En Estados Unidos, el modelo más destacado es el Toyota Mirai, que en 2024 tenía un precio base de 67.095 dólares en California. ([electrive.com](https://www.electrive.com/es/2024/02/27/toyota-opta-por-rebajas-masivas-para-los-coches-de-hidrogeno-en-ee-uu/?utm_source=openai)) Sin embargo, debido a las bajas ventas, Toyota ha implementado descuentos significativos, llegando hasta 40.000 dólares en algunas regiones, reduciendo el precio final a aproximadamente 27.095 dólares. ([electrive.com](https://www.electrive.com/es/2024/02/27/toyota-opta-por-rebajas-masivas-para-los-coches-de-hidrogeno-en-ee-uu/?utm_source=openai))\n\nAdemás de los descuentos, Toyota ofrece incentivos adicionales, como 15.000 dólares en combustible de hidrógeno durante seis años, lo que puede ser atractivo para los consumidores. ([mot

In [21]:
// Join keywords with audit
let joined = sample.map((kwd: any, index: number) => ({
    ...kwd,
    ...auditResults[index]
}));

In [22]:
joined[1]

{
  ROWID: 1381,
  keyword: "peugeot 108 diesel",
  avgMonthlySearches: 20,
  competition: "HIGH",
  competitionIndex: 96,
  averageCpc: 0.8115630149841309,
  lowTopOfPageBid: 0.162650004029274,
  highTopOfPageBid: 0.8118169903755188,
  searchVolume: [
    20, 20, 20, 20, 20,
    20, 20, 20, 20, 10,
    20, 20
  ],
  searchVolumeDate: [
    "2024-10-01T00:00:00.000Z",
    "2024-11-01T00:00:00.000Z",
    "2024-12-01T00:00:00.000Z",
    "2025-01-01T00:00:00.000Z",
    "2025-02-01T00:00:00.000Z",
    "2025-03-01T00:00:00.000Z",
    "2025-04-01T00:00:00.000Z",
    "2025-05-01T00:00:00.000Z",
    "2025-06-01T00:00:00.000Z",
    "2025-07-01T00:00:00.000Z",
    "2025-08-01T00:00:00.000Z",
    "2025-09-01T00:00:00.000Z"
  ],
  searchVolumeGrowthYoy: 0,
  searchVolumeGrowth3m: 100,
  searchVolumeGrowth1m: 0,
  searchVolumeTrend: -0.01276983879506588,
  concepts: [ "motor", "peugeot 108", "diesel" ],
  conceptGroups: [ "Pieza Del Vehículo", "Modelo", "Otros" ],
  prompt: "¿Existe una versión die

In [23]:
await Deno.writeTextFile("./audit.json", JSON.stringify(joined, null, 2));

# Enrich audit

In [24]:
// Dynamically re-import enrichAudit
const { enrichAudit: enrichAuditFresh } = await import("../src/audit.ts");

const cached = JSON.parse(await Deno.readTextFile("./cache.json"));
const result = JSON.parse(await Deno.readTextFile("./audit.json"));
const enrichedAudit = await enrichAuditFresh(result, cached.funnel, cached.personas);

askOpenAISafe retrying! Previous attempt 1 failed with: Previous attempt failed with error: [
  {
    "code": "custom",
    "message": "Subtopic 'Product / Microsite' is not a valid subtopic for topic 'Commercial'. Allowed subtopics are: E-Commerce / Retailer, Marketplace, Affiliate / Comparison Site, Classifieds / Listings, Travel / Booking Platform, Job Board / Recruitment.",
    "path": [
      "subtopic"
    ]
  }
]
Identifying topics...
Assigning topics...
askOpenAISafe retrying! Previous attempt 1 failed with: Previous attempt failed with error: [
  {
    "code": "custom",
    "message": "Subtopic 'Cita y atención al cliente' is not a valid subtopic for topic 'Peugeot'. Allowed subtopics are: Modelos Peugeot eléctricos, Modelos Peugeot híbridos, Modelos Peugeot convencionales, Servicios y mantenimiento Peugeot, Accesorios y repuestos Peugeot.",
    "path": [
      "subtopic"
    ]
  }
]
Assigning funnel stages...
Classifying prompts into personas...
Classifying prompt intents...


In [25]:
enrichedAudit[0]

{
  ROWID: 2291,
  keyword: "coches de hidrógeno precio",
  avgMonthlySearches: 50,
  competition: "MEDIUM",
  competitionIndex: 48,
  averageCpc: 0.32190701365470886,
  lowTopOfPageBid: 0.050508998334407806,
  highTopOfPageBid: 0.7048799991607666,
  searchVolume: [
    50, 70, 50, 70, 90,
    50, 50, 40, 50, 50,
    20, 40
  ],
  searchVolumeDate: [
    "2024-10-01T00:00:00.000Z",
    "2024-11-01T00:00:00.000Z",
    "2024-12-01T00:00:00.000Z",
    "2025-01-01T00:00:00.000Z",
    "2025-02-01T00:00:00.000Z",
    "2025-03-01T00:00:00.000Z",
    "2025-04-01T00:00:00.000Z",
    "2025-05-01T00:00:00.000Z",
    "2025-06-01T00:00:00.000Z",
    "2025-07-01T00:00:00.000Z",
    "2025-08-01T00:00:00.000Z",
    "2025-09-01T00:00:00.000Z"
  ],
  searchVolumeGrowthYoy: -20,
  searchVolumeGrowth3m: -20,
  searchVolumeGrowth1m: 100,
  searchVolumeTrend: -0.053946055471897125,
  concepts: [ "carros" ],
  conceptGroups: [ "No-Marcas", "Otros" ],
  prompt: "¿Cuánto cuestan los coches de hidrógeno?",
  an

In [26]:
await Deno.writeTextFile("./audit.json", JSON.stringify(enrichedAudit, null, 2));

# More enrichments

In [27]:
const result = JSON.parse(await Deno.readTextFile("./audit.json"));

In [ ]:
start = Date.now();

const answers = result.map((response: Record<string, unknown>) => response.answer);
const abss = await extractABSForBrandBatch(answers, cached.brand, 'gpt-4.1');

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`\n✅ Extracted aspect-based sentiments for ${abss.length} texts in ${duration}s\n`);


✅ Extracted aspect-based sentiments for 400 texts in 20.2s



In [29]:
abss[2]

[]

In [30]:
import { extractEntitiesBatch } from "../src/entities.ts";

const entityDefinitions = {
    "brands": "Any brand or companies mentioned",
    "products": "Any products or services mentioned",
    "features": "Specific features or attributes of products/services",
    "issues": "Problems or issues mentioned",
}

start = Date.now();

const answers = result.map((response: Record<string, unknown>) => response.answer);
const entities = await extractEntitiesBatch(answers, entityDefinitions, 'gpt-4.1');

duration = ((Date.now() - start) / 1000).toFixed(1);
console.log(`\n✅ Extracted entities for ${entities.length} texts in ${duration}s\n`);


✅ Extracted entities for 400 texts in 28.7s



In [31]:
entities[0]

[
  { name: "toyota", type: "brand" },
  { name: "mirai", type: "product" },
  { name: "hidrógeno", type: "product" },
  { name: "coche de hidrógeno", type: "product" },
  { name: "vehículo convencional", type: "product" },
  { name: "pila de combustible de hidrógeno", type: "product" },
  { name: "emisión cero", type: "feature" },
  { name: "tiempo de repostaje rápido", type: "feature" },
  { name: "precio elevado", type: "issue" },
  { name: "baja venta", type: "issue" },
  { name: "precio del hidrógeno alto", type: "issue" }
]

In [33]:
// Add sentiment and entity enrichment to audit results
result.forEach((r, i) => {
    r.sentiments = abss[i];
    r.entities = entities[i];
});

await Deno.writeTextFile("./audit.json", JSON.stringify(result, null, 2));

# Read audit parquet

In [ ]:
import { readParquetRows } from "./readParquetRows.ts";

const auditpqt = await readParquetRows<KeywordRecord>("./audit.parquet");
auditpqt[0]

{
  ROWID: 2291n,
  keyword: "coches de hidrógeno precio",
  avgMonthlySearches: 50,
  competition: "MEDIUM",
  competitionIndex: 48,
  averageCpc: 0.32190701365470803,
  lowTopOfPageBid: 0.050508998334407,
  highTopOfPageBid: 0.704879999160766,
  searchVolume: {
    list: [
      { element: 50n },
      { element: 70n },
      { element: 50n },
      { element: 70n },
      { element: 90n },
      { element: 50n },
      { element: 50n },
      { element: 40n },
      { element: 50n },
      { element: 50n },
      { element: 20n },
      { element: 40n }
    ]
  },
  searchVolumeDate: {
    list: [
      { element: 2024-10-01T00:00:00.000Z },
      { element: 2024-11-01T00:00:00.000Z },
      { element: 2024-12-01T00:00:00.000Z },
      { element: 2025-01-01T00:00:00.000Z },
      { element: 2025-02-01T00:00:00.000Z },
      { element: 2025-03-01T00:00:00.000Z },
      { element: 2025-04-01T00:00:00.000Z },
      { element: 2025-05-01T00:00:00.000Z },
      { element: 2025-06-01T00:0

In [43]:
auditpqt[0].searchVolume[0]

In [44]:
auditpqt[0].searchVolumeDate.list[0].element

2024-10-01T00:00:00.000Z

# Google Autocomplete

In [1]:
import { autocomplete, autocompleteSafe, recurseAutocomplete } from "../src/autocomplete.ts";

In [20]:
await autocompleteSafe({query: "coches eléctricos", countryCode: "ES", language: "es"});

[
  "coches eléctricos segunda mano",
  "coches eléctricos chinos",
  "coches eléctricos con mayor autonomía",
  "coches eléctricos precios",
  "coches electricos pequeños",
  "coches eléctricos toyota",
  "coches eléctricos alquiler madrid",
  "coches eléctricos byd",
  "coches eléctricos chinos baratos",
  "coches electricos niños",
  "coches electricos para niños",
  "coches eléctricos para niños 24v baratos",
  "coches eléctricos sin carnet",
  "coches eléctricos más baratos",
  "coches eléctricos madrid"
]

In [3]:
let suggestions = await recurseAutocomplete({
    query: "coches eléctricos",
    countryCode: "ES",
    language: "es",
    maxDepth: 2,
    delayBetweenCalls: 1000
});

suggestions

[
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos segunda mano",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos chinos",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos con mayor autonomía",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos precios",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches electricos pequeños",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos toyota",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos alquiler madrid",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos byd",
    depth: 0
  },
  {
    sourceQuery: "coches eléctricos",
    suggestion: "coches eléctricos chinos baratos",
    depth: 0
  },
  {
    sourceQuery: "coch

In [7]:
const uniqueSuggestions = [...new Set(suggestions.map(s => s.suggestion))];
console.log(uniqueSuggestions.length);
console.log(JSON.stringify(uniqueSuggestions, null, 2));

189
[
  "coches eléctricos segunda mano",
  "coches eléctricos chinos",
  "coches eléctricos con mayor autonomía",
  "coches eléctricos precios",
  "coches electricos pequeños",
  "coches eléctricos toyota",
  "coches eléctricos alquiler madrid",
  "coches eléctricos byd",
  "coches eléctricos chinos baratos",
  "coches electricos niños",
  "coches electricos para niños",
  "coches eléctricos para niños 24v baratos",
  "coches eléctricos sin carnet",
  "coches eléctricos más baratos",
  "coches eléctricos madrid",
  "coches eléctricos segunda mano madrid",
  "coches eléctricos segunda mano alemania",
  "coches eléctricos segunda mano milanuncios",
  "coches eléctricos segunda mano mallorca",
  "coches eléctricos segunda mano barcelona",
  "coches eléctricos segunda mano valencia",
  "coches eléctricos segunda mano galicia",
  "coches eléctricos segunda mano tenerife",
  "coches electricos segunda mano canarias",
  "coches eléctricos segunda mano las palmas",
  "coches electricos segund